In [ ]:
import configparser
import os.path as op

CP = configparser.ConfigParser()
CP.read_file(open(op.join(op.expanduser('~'), '.aws', 'credentials')))
CP.sections()
aws_access_key = CP.get('hcp', 'AWS_ACCESS_KEY_ID')
aws_secret_key = CP.get('hcp', 'AWS_SECRET_ACCESS_KEY')

def attach_keys(arr):
    return [(e, aws_access_key, aws_secret_key) for e in arr]

In [ ]:
def afq_hcp_retest(args):
    import fury
    import xvfbwrapper
    import logging
    import os.path as op
    import s3fs

    from AFQ.data import fetch_hcp
    import AFQ.api as api
    import AFQ.mask as afm
    
    do_single_shell = False
    do_multi_shell = True

    logging.basicConfig(level=logging.INFO)
    log = logging.getLogger(__name__)  
    
    fs = s3fs.S3FileSystem()
    
    subject = args[0]
    aws_access_key = args[1]
    aws_secret_key = args[2]

    _, hcp_1200_bids = fetch_hcp(
        [subject],
        profile_name=False,
        study='HCP_1200',
        aws_access_key_id=aws_access_key,
        aws_secret_access_key=aws_secret_key)

    _, hcp_retest_bids = fetch_hcp(
        [subject],
        profile_name=False,
        study='HCP_Retest',
        aws_access_key_id=aws_access_key,
        aws_secret_access_key=aws_secret_key)

    if do_single_shell:
        myafq_1200_single_shell = api.AFQ(
            hcp_1200_bids,
            brain_mask=afm.LabelledMaskFile('seg', scope='dmriprep', exclusive_labels=[0]),
            min_bval=990,
            max_bval=1010,
            virtual_frame_buffer=True)

        myafq_retest_single_shell = api.AFQ(
            hcp_retest_bids,
            brain_mask=afm.LabelledMaskFile('seg', scope='dmriprep', exclusive_labels=[0]),
            min_bval=990,
            max_bval=1010,
            virtual_frame_buffer=True)

        myafq_1200_single_shell.export_all()
        fs.put(op.join(hcp_1200_bids, "derivatives/afq"), "profile-hcp/hcp_reliability/single_shell/hcp_1200", recursive=True)

        myafq_retest_single_shell.export_all()
        fs.put(op.join(hcp_retest_bids, "derivatives/afq"), "profile-hcp/hcp_reliability/single_shell/hcp_retest", recursive=True)

    if do_multi_shell:
        myafq_1200_multi_shell = api.AFQ(
            hcp_1200_bids,
            brain_mask=afm.LabelledMaskFile('seg', scope='dmriprep', exclusive_labels=[0]),
            tracking_params = {"odf_model": "DKI", 'seed_mask': afm.ScalarMask('dki_fa'), 'stop_mask': afm.ScalarMask('dki_fa')},
            scalars=["dki_fa", "dki_md"],
            force_recompute=do_single_shell,
            virtual_frame_buffer=True)

        myafq_retest_multi_shell = api.AFQ(
            hcp_retest_bids,
            brain_mask=afm.LabelledMaskFile('seg', scope='dmriprep', exclusive_labels=[0]),
            tracking_params = {"odf_model": "DKI", 'seed_mask': afm.ScalarMask('dki_fa'), 'stop_mask': afm.ScalarMask('dki_fa')},
            scalars=["dki_fa", "dki_md"],
            force_recompute=do_single_shell,
            virtual_frame_buffer=True)

        myafq_1200_multi_shell.export_all()
        fs.put(op.join(hcp_1200_bids, "derivatives/afq"), "profile-hcp/hcp_reliability/multi_shell/hcp_1200", recursive=True)

        myafq_retest_multi_shell.export_all()
        fs.put(op.join(hcp_retest_bids, "derivatives/afq"), "profile-hcp/hcp_reliability/multi_shell/hcp_retest", recursive=True)


In [ ]:
subjects_num = [
    169343,
    177746,
    195041,
#     185442,
#     433839,
#     144226,
#     200614,
#     204521,
#     601127,
#     877168,
]
subjects = [str(i) for i in subjects_num] 
args = attach_keys(subjects)

In [ ]:
#afq_hcp_retest(attach_keys(['169343'])[0])

In [ ]:
import cloudknot as ck
ck.set_region('us-east-1')

In [ ]:
knot = ck.Knot(name='hcp_api-64gb-20821-0',
              func=afq_hcp_retest,
              base_image='libglxvfb:1',
              image_github_installs="https://github.com/36000/pyAFQ.git@mask_files",
              pars_policies=('AmazonS3FullAccess',),
              bid_percentage=100,
              volume_size=64,
              memory=64000)

In [ ]:
result_futures = knot.map(args)

In [ ]:
ck.set_region('us-east-1')
knot.view_jobs()

In [ ]:
j0 = knot.jobs[0]
j0.status

In [ ]:
knot.clobber(clobber_pars=True, clobber_repo=True, clobber_image=True)

In [ ]:
import AFQ.api as api
from importlib import reload
reload(api)

In [ ]:
api.download_and_combine_afq_profiles(
    "~/hcp_reliability_profiles/single_shell_test_profiles.csv",
    "profile-hcp",
    "hcp_reliability/single_shell/hcp_1200"
    )
api.download_and_combine_afq_profiles(
    "~/hcp_reliability_profiles/single_shell_retest_profiles.csv",
    "profile-hcp",
    "hcp_reliability/single_shell/hcp_retest"
    )
api.download_and_combine_afq_profiles(
    "~/hcp_reliability_profiles/multi_shell_test_profiles.csv",
    "profile-hcp",
    "hcp_reliability/multi_shell/hcp_1200"
    )
api.download_and_combine_afq_profiles(
    "~/hcp_reliability_profiles/multi_shell_retest_profiles.csv",
    "profile-hcp",
    "hcp_reliability/multi_shell/hcp_retest"
    )

In [ ]:
import AFQ.viz.utils as vut
import logging
single_comparisons = vut.LongitudinalCSVComparison(
    'hcp_reliability_profiles/comparisons',
    [
        "~/hcp_reliability_profiles/single_shell_test_profiles.csv",
        "~/hcp_reliability_profiles/single_shell_retest_profiles.csv"
    ],
    ['single_test', 'single_retest'])
multi_comparisons = vut.LongitudinalCSVComparison(
    'hcp_reliability_profiles/comparisons',
    [
        "~/hcp_reliability_profiles/multi_shell_test_profiles.csv",
        "~/hcp_reliability_profiles/multi_shell_retest_profiles.csv"
    ],
    ['multi_test', 'multi_retest'],
    scalar_bounds={'lb': {'dki_fa': 0.2},
                   'ub': {'dki_md': 0.002}})

single_comparisons.logger.setLevel(logging.WARNING)
multi_comparisons.logger.setLevel(logging.WARNING)

In [ ]:
single_comparisons.reliability_plots(ylims=[0.0, 1.0], show_plots=True)
multi_comparisons.reliability_plots(ylims=[0.0, 1.0], scalars=["dki_fa", "dki_md"], show_plots=True)